## Algorithm の選択

In [7]:
# 余分な Warning を非表示にする
import warnings

warnings.filterwarnings('ignore')

# Library の Import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# matplotlib 日本語化対応
import japanize_matplotlib

# 表示 Option の調整
np.set_printoptions(suppress=True, precision=4)  # NumPy の浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format  # pandas での浮動小数点の表示精度
pd.set_option('display.max_columns', None)  # DataFrame ですべての項目を表示
plt.rcParams['font.size'] = 14  # Graph の Default font 指定
random_seed = 123  # 乱数の種

In [2]:
# Sample data の読み込み
# （乳がん疾患 Data）

# Data の load
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()

# 入力 Data: x(30次元)
# 正解 Data: y
x = cancer.data
y = cancer.target

# Sample data の分割

# Data 分割の Parameters
test_size = 0.1

# Data 分割
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=random_seed, stratify=y)

# 分割 Size 確認
print(x.shape)
print(x_train.shape)
print(x_test.shape)

(569, 30)
(512, 30)
(57, 30)


## 複数 Algorithm の List を作成

In [3]:
# 複数 Algorithm で精度を比較
# 結果が同じになるよう random_state は同一にする

# 線形回帰
from sklearn.linear_model import LogisticRegression

algorithm = LogisticRegression(random_state=random_seed)

# Support vector machine(Kernel)
from sklearn.svm import SVC

algorithm2 = SVC(kernel='rbf', random_state=random_seed)

# 決定木
from sklearn.tree import DecisionTreeClassifier

algorithm3 = DecisionTreeClassifier(random_state=random_seed)

# Random forest
from sklearn.ensemble import RandomForestClassifier

algorithm4 = RandomForestClassifier(random_state=random_seed)

# XGBoost
from xgboost import XGBClassifier

algorithm5 = XGBClassifier(random_state=random_seed)

# Algorithm の List 作成
algorithms = [algorithm, algorithm2, algorithm3, algorithm4, algorithm5]

In [4]:
# 複数 Algorithm で精度比較
for algorithm in algorithms:
    # 訓練 Data で学習
    algorithm.fit(x_train, y_train)
    # 検証 Data で精度測定
    score = algorithm.score(x_test, y_test)
    # Algorithm 名取得
    name = algorithm.__class__.__name__
    # 精度と Algorithm 名表示
    print(f'score: {score:.4f} {name}')

score: 0.9474 LogisticRegression
score: 0.8947 SVC
score: 0.9474 DecisionTreeClassifier
score: 0.9298 RandomForestClassifier
[05:18:24] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
score: 0.9825 XGBClassifier


- Support Vector Machine は、精度が悪い
- XGBoost は精度が良く検証 Data に対して98.25% となっている
- 他の３つの候補は似たり寄ったりだが、Logistic 回帰と決定木の精度が Random Forest よりは良い。

この見た目の結論は、本当に正しいのか？

## Hyper Parameters の最適化
### Support Vector Machine の Parameters 値
- gamma: 'scale'
- C: 1.0

In [5]:
# gamma の最適化
gammas = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]

for gamma in gammas:
    algorithm = SVC(kernel='rbf', gamma=gamma, random_state=random_seed)
    algorithm.fit(x_train, y_train)
    score = algorithm.score(x_test, y_test)
    print(f'score: {score:.4f} gamma: {gamma}')

score: 0.6316 gamma: 1
score: 0.6316 gamma: 0.1
score: 0.6316 gamma: 0.01
score: 0.9474 gamma: 0.001
score: 0.9474 gamma: 0.0001
score: 0.9474 gamma: 1e-05


上記の結果から、0.001 を採用。

###### gamma の tuning 方法
１を出発点に、順に 1/10 倍して値を小さくし、最適な値を見つける。

In [6]:
# C の最適化
# gamma は、先ほど調べた最適値 0.001 を採用

Cs = [1, 10, 100, 1000, 10000]
for C in Cs:
    algorithm  = SVC(kernel='rbf', gamma=0.001, C=C, random_state=random_seed)
    algorithm.fit(x_train, y_train)
    score = algorithm.score(x_test, y_test)
    print(f'score: {score:.4f} C: {C}')

score: 0.9474 C: 1
score: 0.9298 C: 10
score: 0.9298 C: 100
score: 0.9298 C: 1000
score: 0.9298 C: 10000


C 値に関しては、 Default 値の１が最適であることを確認。

##### C 値の tuning 方法
Default 値の１を出発点に、10倍ずつしていって最適値を求める。

## 公差検定法

In [8]:
# 特定の Algorithm に対して公差検定を実施

# Algorithm の定義
algorithm = SVC(kernel='rbf', random_state=random_seed, gamma=0.001, C=1)

# 分割時に正解 Data の分布が偏らないように StratifiedKFold を利用
stratifiedkfold = StratifiedKFold(n_splits=3)

# 公差検定の実施（分割数=3）
scores = cross_val_score(algorithm, x_train, y_train, cv=stratifiedkfold)

# 平均値の計算
mean = scores.mean()

# 結果表示
print(f'平均 Score: {mean:.4f} 個別 Score: {scores}')

平均 Score: 0.9141 個別 Score: [0.8889 0.9181 0.9353]


```
from sklearn.model_selection import cross_val_score
```
scikit-learn の cross_val_score 関数を利用することで公差検定を行なえる。
- 引数: algorithm x, y, 分割数 cv を指定
- 戻り値: 分割数と同じ要素数の NumPy 配列（※上記では、３つの Score の平均値を mean()関数で求めて合わせて表示）